<a href="https://colab.research.google.com/github/yonseimath/datascience-biginner-2022-kaggle-competitions/blob/feature%2Fjuwon/juwon/220703%20%EC%9E%90%EC%97%B0%EC%96%B4(Getting%20Started%20with%20AI4Code)%20%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/choione/getting-started-with-ai4code-study


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.1  # size of validation set 
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)

# Split, keeping notebooks with a common origin (ancestor_id) together

ids = df.index.unique('id')
ancestors = df_ancestors.loc[ids, 'ancestor_id']
ids_train, ids_valid = next(splitter.split(ids, groups = ancestors))
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

NVALID = 0.1 

: float인 경우 0.0에서 1.0 사이여야 하며 테스트 분할(반올림)에 포함할 그룹의 비율을 나타냅니다. int이면 테스트 그룹의 절대 수를 나타냅니다. None인 경우 값은 열차 크기의 보수로 설정됩니다. 기본값은 버전 0.21에서 변경됩니다. 가 지정되지 않은 경우에만 0.2로 유지 train_size되고, 그렇지 않으면 지정된 train_size.

n_splits=1
: 정수, 기본값=5
다시 섞기 및 분할 반복 횟수입니다.

random_state=0
: int, RandomState 인스턴스 또는 없음, 기본값=없음
생성된 훈련 및 테스트 인덱스의 무작위성을 제어합니다.

Split, keeping notebooks with a common origin (ancestor_id) together
:# 공통 출처(ancestor_id)를 가진 노트북을 함께 유지

**피처 엔지니어링**¶ 순위 모델과 함께 사용할 tf-idf 기능을 생성해 보겠습니다. 이러한 기능은 모델이 노트북 내 다양한 ​​위치에서 가장 자주 발생하는 경향이 있는 단어 종류를 학습하는 데 도움이 됩니다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Training set
tfidf = TfidfVectorizer(min_df=0.01)
X_train = tfidf.fit_transform(df_train['source'].astype(str))
# Rank of each cell within the notebook, 노트북 내 각 셀의 순위
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook, 각 노트북의 셀 수
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

sklearn.feature_extraction.text: 원시 문서 모음을 TF-IDF 기능의 매트릭스로 변환합니다.

min_df=0.01

min_df: float or int, default=1 

: [0.0, 1.0] 범위의 float인 경우 매개변수는 문서의 비율, 정수 절대 개수를 나타냅니다. 어휘가 없음이 아닌 경우 이 매개변수는 무시됩니다.

X_train = tfidf.fit_transform(df_train['source'].astype(str)) 

y_train = df_ranks.loc[ids_train].to_numpy()

X: 배열과 유사한 모양(n_samples, n_features)
훈련 데이터, 여기서 n_samples는 샘플 수이고 n_features는 기능 수입니다.

y: 배열과 유사한 모양(n_samples,), 기본값=없음
지도 학습 문제의 대상 변수입니다.

roups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

그룹: 배열과 유사한 모양(n_samples,)
데이터 세트를 훈련/테스트 세트로 분할하는 동안 사용된 샘플의 그룹 레이블입니다.

Now let's add the code cell ordering as a feature. We'll append a column that enumerates the code cells in the correct order, like 1, 2, 3, 4, ..., while having the dummy value 0 for all markdown cells. This feature will help the model learn to put the code cells in the correct order.

In [ ]:
# Add code cell ordering
X_train = sparse.hstack((
    X_train,
    np.where(
        df_train['cell_type'] == 'code',
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))
print(X_train.shape)

몇번째 중복된 데이터인지 확인하기: cumcount가 0인 경우 중복되지 않음

: df.groupby('이름', as_index=False).cumcount()


dataFrame 적용 가능 메서드: to_numpy, values 

.to_numpy() 메서드

:이 메서드는 pandas 객체를 numpy 배열 객체인 ndarray로 반환한다.

Series가 ndarray-like이지만 ndarray가 아니므로 실제 ndarray가 필요한 상황이 있을 수 있다.

이런 경우에는 to_numpy() 메서드를 사용하면 된다.

.reshape(-1, 1)

: 파이썬 NumPy 에서 배열의 차원(Dimension)을 재구조화, 변경하고자 할 때 reshape() 메소드를 사용합니다.  가령, 3개의 행과 4개의 열로 구성된 2차원의 배열로 재설정하고 싶으면 reshape(3, 4) 처럼 reshape()의 매개변수로 변경하고자 하는 배열의 행과 열의 차원을 정수로 입력해주면 됩니다. 

그런데 reshape(-1, 2) 혹은 reshape(3, -1) 처럼 reshape() 메소드 안에 '-1'이 들어가 있는 경우가 있습니다. 이때 reshape()의 '-1'이 의미하는 바는, 변경된 배열의 '-1' 위치의 차원은 "원래 배열의 길이와 남은 차원으로 부터 추정"이 된다는 뜻입니다.

** reshape(-1, 정수) 의 행(row) 위치에 '-1'이 들어있을 경우**

12개의 원소가 있을 때:

배열 x에 대해서 x.reshape(-1, 정수) 를 해주면 '열(column)' 차원의 '정수'에 따라서 12개의 원소가 빠짐없이 배치될 수 있도록 '-1'이 들어가 있는 '행(row)' 의 개수가 가변적으로 정해짐을 알 수 있습니다.  

x.reshape(-1, 2) => shape(6, 2), x.reshape(-1, 3)=> shape(4, 3) ... 


**Train¶**

In [ ]:
from xgboost import XGBRanker

model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)
model.fit(X_train, y_train, group=groups)

현존 머신러닝 모델중 가장 우월한 XGBoost 모델

이 알고리즘을 병렬 학습이 지원되도록 구현한 라이브러리가 XGBoost 이다. Regression, Classification 문제를 모두 지원하며, 성능과 자원 효율이 좋아서, 인기 있게 사용되는 알고리즘 이다.

min_child_weight=10

자식에게 필요한 인스턴스 가중치(헤시안)의 최소 합계입니다. 트리 분할 단계에서 인스턴스 가중치의 합이 보다 작은 리프 노드가 min_child_weight생성되면 빌드 프로세스에서 추가 분할을 포기합니다. 

선형 회귀 작업에서 이는 단순히 각 노드에 있어야 하는 최소 인스턴스 수에 해당합니다. 클수록 min_child_weight알고리즘이 더 보수적입니다.

,subsample=0.5

subsample[기본값=1]
: 훈련 인스턴스의 하위 샘플 비율입니다. 0.5로 설정하면 XGBoost가 나무를 성장시키기 전에 훈련 데이터의 절반을 무작위로 샘플링합니다. 

그리고 이것은 과적합을 방지할 것입니다. 서브샘플링은 모든 부스팅 반복에서 한 번 발생합니다.

tree_method='hist',

tree_method문자열 [기본값= auto]: 
XGBoost에서 사용되는 트리 구성 알고리즘입니다.

hist: 더 빠른 히스토그램 최적화된 근사 욕심 알고리즘.

model.fit은 model.compile에서 지정한 방식으로 학습을 진행합니다.

**Evaluate**

**Validation set**

먼저 훈련 세트와 마찬가지로 검증 세트에 대한 기능을 생성합니다.

In [ ]:
# Validation set
X_valid = tfidf.transform(df_valid['source'].astype(str))
# The metric uses cell ids
y_valid = df_orders.loc[ids_valid]

X_valid = sparse.hstack((
    X_valid,
    np.where(
        df_valid['cell_type'] == 'code',
        df_valid.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

validation set을 사용하는 이유는 간단합니다. 바로 "모델의 성능을 평가하기 위해서" 입니다.

첫 번째는 test accuracy를 가늠,정확도를 측정. 
두 번째는 모델을 튜닝하여 모델의 성능을 높일 수 있습니다.

astype(str) << 데이터 프레임 타입 바꾸기. 

hstack에서 앞의 h는 Horizontal 즉 수평이라는 뜻으로 hstack을 사용하면 가로로 행렬 결합이 이뤄집니다.파라미터는 hstack() 괄호 안에 합치고 싶은 행렬들을 넣으면 되는데 다만 주의해야할 점은 합쳐질 행렬들이 행이나 열의 개수가 일치가 되어야 합니다.그리고 hstack을 사용할 때는 반드시 요소들의 (행) 개수가 일치해야 하며 배열의 열 개수는 일치하지 않아도 가능합니다.

여기에서 모델을 사용하여 노트북 내 각 셀의 순위를 예측한 다음 이 순위를 정렬된 셀 ID 목록으로 변환합니다.↓

In [ ]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank. 각 노트북의 셀을 순위별로 정렬합니다.
                                  # The cell_ids are now in the order the model predicted. cell_ids는 이제 모델이 예측한 순서대로 있습니다.
    .reset_index('cell_id')  # Convert the cell_id index into a column. cell_id 인덱스를 열로 변환합니다.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list. 각 노트북의 cell_ids를 목록으로 그룹화하십시오.
)
y_pred.head(10)

이제 노트북을 살펴보고 모델이 어떻게 되었는지 살펴보겠습니다.

In [ ]:
nb_id = df_valid.index.get_level_values('id').unique()[8]

display(df.loc[nb_id])
display(df.loc[nb_id].loc[y_pred.loc[nb_id]])

get_level_values ​​( 수준 )
: 요청된 수준에 대한 값의 인덱스를 반환합니다.

unique()는 데이터에 고유값들이 어떠한 종류들이 있는지 알고 싶을때 사용하는 함수입니다. 데이터의 종류들을 알고 싶을때 사용하면 유용한 함수입니다.